# Playlist recommender using Locality-Sensitive Hashing and Jaccard similarity

In [11]:
import pandas as pd
import numpy as np
from math import log2
import random

##  Load train data

We begin by loading the training dataset, which contains playlists and their associated tracks.  

In [12]:

df = pd.read_csv("train.csv")
df.head()


,playlist_id,playlist_name,artist_id,artist_name,track_id,track_name,album_id,album_name,duration_ms,position,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,461206,Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,Miranda Lambert,4Gyhy413uPALzaVg4S1DpX,Keeper of the Flame,563h536tB6n8Dn62jr4RZG,The Weight of These Wings,239733.0,95.0,...,6.0,-6.372,1.0,0.0361,0.4500,0.000023,0.1300,0.337,116.567,4.0
1,545224,mhm,3zLWOB0I86EiVgG5NrX1ht,Jarrod Alonge,3jKrgzpPb23ZmEIAipUKnZ,"Hey Jarrod, What's That Song Again?",0XFMnZrWpEYMVRrBdHmsGZ,Beating a Dead Horse: Deluxe Ultra-Limited Exc...,255185.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0dMexqq0XIWS3QJ74z3ZhD,Hip Hop 2000s Music - Best Hip Hop Hits of the...,NaN,NaN,5dL5jv5GSCRoDhTtnY8maL,Mesmerize,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,279103,litty,46SHBwWsqBkxI7EeeBEQG7,Kodak Black,34oWbFBfGEElvgO0a5c9V4,No Flockin,12YTH28wiBXQ16gvWOCMLU,No Flockin,165290.0,18.0,...,5.0,-8.372,0.0,0.1910,0.0673,0.000000,0.0839,0.815,117.532,4.0
4,1WH6WVBwPBz35ZbWsgCpgr,Top Pop Hits 2015-2025,NaN,"Topic, A7S",3H7ihDc1dqLriiWXwsc2po,Breaking Me,NaN,NaN,166794.0,NaN,...,8.0,-5.652,0.0,0.2180,0.2230,0.000000,0.1290,0.664,122.031,4.0


### Group tracks by playlist

To work with playlists as sets, we group all track IDs belonging to the same playlist. 
Each playlist is therefore represented as a Python set, where duplicate tracks are removed automatically.  

In [13]:

playlist_dict = (
    df.groupby("playlist_id")["track_id"]
      .apply(set)
      .to_dict()
)

# make sure playlist_id are strings 
playlist_dict = {
    str(pid): tracks
    for pid, tracks in playlist_dict.items()
}

sizes = {pid: len(s) for pid, s in playlist_dict.items()}
min(sizes.values()), max(sizes.values())

(1, 439)

## MinHash signatures 

Each playlist is first transformed into a MinHash signature—a compact representation that preserves similarity.  
Playlists with similar signatures have a high probability of being placed in the same LSH buckets

MinHash works by hashing each element in a set using a family of independent hash functions and keeping only the minimum hashed value for each function. 

A key detail is that all MinHash instances must share the exact same hash functions. Otherwise, signatures become incomparable and LSH cannot identify similar playlists. We therefore generate a fixedhash functions reuse them for all playlists.

Python’s built-in `hash()` cannot be used because it changes across runs, so we define `stable_hash` function to ensure reproducible hashing. Using these components, each playlist is converted into a compact MinHash signature that captures the identity of its track set while enabling efficient comparison through LSH.





In [14]:

NUM_PERM = 256
MAX_HASH = 2**32 - 1

def generate_hash_params(num_perm, seed=42):
    random.seed(seed)

    prime = 4294967311
    params = []
    for _ in range(num_perm):
        a = random.randint(1, prime - 1)
        b = random.randint(0, prime - 1)
        params.append((a, b, prime))
    return params

# IMPORTANT: all MinHash objects must share the same hash functions
GLOBAL_HASH_PARAMS = generate_hash_params(NUM_PERM, MAX_HASH)

def stable_hash(x):
    return int.from_bytes(x.encode('utf8'), 'little') & 0xffffffff

class MinHash:
    def __init__(self, num_perm=512):
        self.num_perm = num_perm
        self.max_hash = (1 << 32) - 1
        self.hash_params = GLOBAL_HASH_PARAMS   # shared
        self.signature = [self.max_hash] * num_perm

    def update(self, value):
        h = stable_hash(value)
        for i, (a, b, prime) in enumerate(self.hash_params):
            new_hash = (a * h + b) % prime
            if new_hash < self.signature[i]:
                self.signature[i] = new_hash

    def digest(self):
        return self.signature


def create_minhash_from_tracks(track_set):
    m = MinHash()
    for t in track_set:
        m.update(str(t))
    return m



minhash_dict = {
    pid: create_minhash_from_tracks(tracks)
    for pid, tracks in playlist_dict.items()
}

sigs = [mh.digest() for mh in minhash_dict.values()]
unique_sigs = len({tuple(sig) for sig in sigs})
print("Unique signatures:", unique_sigs)
print("Total playlists:", len(minhash_dict))



Unique signatures: 932
Total playlists: 936


### Locality-Sensitive Hashing (LSH) to find neighbors fast

Computing Jaccard similarity between all pairs of playlists is computationally expensive, especially when the dataset grows.  
To address this, we use Locality-Sensitive Hashing (LSH), which enables fast approximate nearest-neighbor search.
The signature of length num_perm is divided into a number of *bands*, each containing a fixed number of rows. For every band, the corresponding slice of the signature is hashed and stored in a dictionary of buckets. Playlists whose band hashes match are placed into the same bucket and then treated as candidate neighbors.

After building the LSH index over all playlists, we can query it with the MinHash signature of any playlist to obtain a list of candidate neighbors. The final line below demonstrates that the indexing and lookup works as expected.


In [15]:
class LSH:
    def __init__(self, num_perm=NUM_PERM, bands=256):
        assert num_perm % bands == 0, "num_perm must be divisible by bands"
        self.num_perm = num_perm
        self.bands = bands
        self.rows = num_perm // bands      # rows per band
        self.buckets = [dict() for _ in range(bands)]

    def _band_hash(self, band_values):
        return hash(tuple(band_values))

    def insert(self, key, minhash_obj):
        sig = minhash_obj.digest()
        for b in range(self.bands):
            start = b * self.rows
            end = start + self.rows
            band = sig[start:end]
            h = self._band_hash(band)
            bucket = self.buckets[b].setdefault(h, [])
            bucket.append(key)

    def query(self, minhash_obj):
        sig = minhash_obj.digest()
        candidates = set()
        for b in range(self.bands):
            start = b * self.rows
            end = start + self.rows
            band = sig[start:end]
            h = self._band_hash(band)
            bucket = self.buckets[b].get(h)
            if bucket:
                candidates.update(bucket)
        return list(candidates)


lsh = LSH(num_perm=NUM_PERM, bands=256)

for pid, mh in minhash_dict.items():
    lsh.insert(pid, mh)

# Testing for a random playlist
some_pid = next(iter(minhash_dict.keys()))
print("Example LSH candidates:", lsh.query(minhash_dict[some_pid])) 





Example LSH candidates: ['7DgPQwzEoUVfQYBiMLER9Z', '4DJztJkufdlND0Hvg4nGkK', '489404', '523500', '5B46dEOFzUu8z3uPC9lBLt', '0dMexqq0XIWS3QJ74z3ZhD', '56un2laj6rmMUKhDlkUkAY']


### Generating recommendations

Once we can retrieve similar playlists through LSH, we generate recommendations by aggregating tracks from the nearest neighbors.  
The intuition is that playlists that share many songs with a query playlist likely contain additional relevant tracks.

For a given playlist:
1. LSH retrieves a set of similar playlists.
2. Tracks from these neighbors are scored based on Jaccard
3. Tracks already present in the playlist are removed.
4. The highest-scoring tracks are returned as recommendations.

This simple neighborhood-based strategy provides a fast and effective baseline recommender system.

In [16]:

def jaccard_similarity(set1, set2):
    if not set1 and not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)


def recommend(pid, top_k=10, top_neighbors=100):
    """
    Recommend tracks for a given playlist ID using Jaccard similarity for scoring.

    """

    # get MinHash signature for the query playlist
    mh = minhash_dict[pid]

    # Retrieve candidate neighbors from LSH buckets
    candidates = lsh.query(mh)

    # Remove itself if present
    candidates = [c for c in candidates if c != pid]

    # Visible tracks of query playlist (if testing)
    # or full playlist if training
    query_tracks = playlist_dict[pid]

    # ---- Score neighbors by Jaccard similarity ----
    neighbor_scores = []
    for c in candidates:
        sim = jaccard_similarity(query_tracks, playlist_dict[c])
        neighbor_scores.append((sim, c))

    # Sort neighbors by similarity
    neighbor_scores.sort(reverse=True)
    top_neighbors = neighbor_scores[:top_neighbors]

   
    scores = {}
    for sim, nbr in top_neighbors:
        for track in playlist_dict[nbr]:
            if track not in query_tracks:
                scores[track] = scores.get(track, 0) + sim

    # Return top-K highest scoring tracks
    ranked = sorted(scores, key=scores.get, reverse=True)
    return ranked[:top_k]



recommendations = recommend(some_pid)
recommendations

['5ByAIlEEnxYdvpnezg7HTX',
 '7kfTqGMzIHFWeBeOJALzRf',
 '4Km5HrUvYTaSUfiSGPJeQR',
 '7dZAPeA3Of5j5Vaef0DQ6M',
 '7frWizM5FgJkaUl1rXhVtO',
 '4Kd0FzFpOgIGxlBl4HXuFn',
 '5zN3VFmNhdOKxRElarvVq5',
 '04KTF78FFg8sOHC1BADqbY',
 '05VHidqx1tV6V7MsCdAIby',
 '2NBQmPrOEEjA8VbeWOQGxO']

In [17]:
def print_playlist_and_recommendations(pid, top_k=10):
    # --- 1. Print current playlist tracks ---
    print(f"\n=== Tracks in Playlist {pid} ===")
    
    playlist_tracks = playlist_dict[pid]
    
    playlist_df = (
        df[df['track_id'].isin(playlist_tracks)]
        [['track_id', 'track_name', 'artist_name']]
        .drop_duplicates()
    )

    for _, row in playlist_df.iterrows():
        print(f"• {row['track_name']} — {row['artist_name']}")

    # --- 2. Compute recommendations ---
    rec_ids = recommend(pid, top_k=top_k)

    print(f"\n=== Recommended Tracks for Playlist {pid} ===")

    if len(rec_ids) == 0:
        print("No recommendations found.")
        return
    
    rec_df = (
        df[df['track_id'].isin(rec_ids)]
        [['track_id', 'track_name', 'artist_name']]
        .drop_duplicates()
    )

    # --- 3. Print recommended tracks ---
    for _, row in rec_df.iterrows():
        print(f"• {row['track_name']} — {row['artist_name']}")

query_pid = list(playlist_dict.keys())[2]
print_playlist_and_recommendations(query_pid, top_k=10)



=== Tracks in Playlist 101861 ===
• Summer — Marshmello
• Where Are Ü Now (with Justin Bieber) - Marshmello Remix — Jack Ü
• Alone — Marshmello

=== Recommended Tracks for Playlist 101861 ===
• Moving On — Marshmello
• Silence — Marshmello
• Home — Marshmello
• Alarm - Marshmello Remix — Anne-Marie
• Fade — Alan Walker
• Waiting For Love - Marshmello Remix — Avicii
• Take It Back — Marshmello


### Evaluation 

To evaluate the effectiveness of the recommendation system, we use a common methodology.
For each test playlist, we hide a portion of the tracks (treated as ground truth) and let the model predict them based only on the visible tracks.

We compute:
- **Precision@k:** How many recommended tracks are correct.
- **Recall@k:** How many of the hidden tracks were recovered.
- **MAP@k:** How well the model ranks relevant tracks.
- **NDCG@k:** Whether relevant tracks appear near the top of the recommendation list.

These metrics allow us to assess both retrieval quality and ranking performance.  
This evaluation setup is consistent with standard recommender system benchmarks, including the Spotify Million Playlist Dataset Challenge.


In [18]:

test_df = pd.read_csv("test.csv")

# Group tracks for each test playlist
playlist_tracks_test = (
    test_df.groupby("playlist_id")["track_id"]
    .apply(list)
    .to_dict()
)

visible_test = {}
hidden_test = {}

hide_ratio = 0.25   # hide 25% of tracks

for pid, tracks in playlist_tracks_test.items():
    if len(tracks) < 3:
        continue

    n_hide = max(1, int(len(tracks) * hide_ratio))
    hidden = set(random.sample(tracks, n_hide))
    visible = set(tracks) - hidden

    hidden_test[pid] = hidden
    visible_test[pid] = visible

test_pids = list(visible_test.keys())

def create_minhash_from_tracks(track_set):
    m = MinHash(num_perm=512)
    for t in track_set:
        m.update(str(t))
    return m

minhash_test = {
    pid: create_minhash_from_tracks(visible)
    for pid, visible in visible_test.items()
}


def jaccard_similarity(set1, set2):
    if not set1 and not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)


def recommend_from_visible(pid, top_k=10, top_neighbors=200):
    mh = minhash_test[pid]
    visible = visible_test[pid]

    # get candidate neighbors from LSH
    candidates = lsh.query(mh)

    # score neighbors by exact Jaccard similarity
    neighbor_scores = []
    for c in candidates:
        base = playlist_dict[c]  # training playlist
        sim = jaccard_similarity(visible, base)
        neighbor_scores.append((sim, c))

    # sort neighbors by Jaccard score
    neighbor_scores.sort(reverse=True)
    top_neighbors = neighbor_scores[:top_neighbors]

    # aggregate track scores
    scores = {}
    for sim, c in top_neighbors:
        for track in playlist_dict[c]:
            if track not in visible:
                scores[track] = scores.get(track, 0) + sim

    # return top-K tracks
    ranked = sorted(scores, key=scores.get, reverse=True)
    return ranked[:top_k]


def precision_at_k(rec, gt, k):
    return len(set(rec[:k]) & gt) / k

def recall_at_k(rec, gt, k):
    return len(set(rec[:k]) & gt) / len(gt)

def average_precision(rec, gt, k):
    score = 0
    hits = 0
    for i, track in enumerate(rec[:k], start=1):
        if track in gt:
            hits += 1
            score += hits / i
    return score / min(len(gt), k)

def ndcg_at_k(rec, gt, k):
    dcg = 0
    for i, track in enumerate(rec[:k], start=1):
        if track in gt:
            dcg += 1 / log2(i + 1)
    ideal = min(len(gt), k)
    idcg = sum(1 / log2(i + 1) for i in range(1, ideal + 1))
    return dcg / idcg if idcg > 0 else 0


K = 10

precisions = []
recalls = []
maps = []
ndcgs = []

for pid in test_pids:
    recs = recommend_from_visible(pid, top_k=K)
    gt = hidden_test[pid]

    precisions.append(precision_at_k(recs, gt, K))
    recalls.append(recall_at_k(recs, gt, K))
    maps.append(average_precision(recs, gt, K))
    ndcgs.append(ndcg_at_k(recs, gt, K))


print("\n=== Evaluation Results (k=10) ===")
print(f"Precision@10: {np.mean(precisions):.4f}")
print(f"Recall@10:    {np.mean(recalls):.4f}")
print(f"MAP@10:       {np.mean(maps):.4f}")
print(f"NDCG@10:      {np.mean(ndcgs):.4f}")



=== Evaluation Results (k=10) ===
Precision@10: 0.0068
Recall@10:    0.0632
MAP@10:       0.0186
NDCG@10:      0.0293
